In [ ]:
import cv2
import numpy as np
import pickle

# Define the Cartoonizer class
class Cartoonizer:
    def __init__(self, line_size=7, blur_value=7, k=9):
        self.line_size = line_size
        self.blur_value = blur_value
        self.k = k

    def edge_mask(self, img):
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray_blur = cv2.medianBlur(gray, self.blur_value)
        edges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                      cv2.THRESH_BINARY, self.line_size, self.blur_value)
        return edges

    def color_quantization(self, img):
        data = np.float32(img).reshape((-1, 3))
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)
        _, label, center = cv2.kmeans(data, self.k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        center = np.uint8(center)
        result = center[label.flatten()]
        result = result.reshape(img.shape)
        return result

    def cartoonize(self, img):
        edges = self.edge_mask(img)
        img = self.color_quantization(img)
        blurred = cv2.bilateralFilter(img, d=3, sigmaColor=200, sigmaSpace=200)
        cartoon = cv2.bitwise_and(blurred, blurred, mask=edges)
        return cartoon

# Instantiate the Cartoonizer class
cartoonizer = Cartoonizer()

# Save the model to a .pkl file
with open('cartoonizer_model.pkl', 'wb') as file:
    pickle.dump(cartoonizer, file)

print("Model saved to cartoonizer_model.pkl")
